# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,299.11,72,17,3.32,PN,1741232385
1,1,bereznik,62.8551,42.7072,270.65,95,100,3.60,RU,1741232401
2,2,palu,-0.8917,119.8707,303.67,75,100,2.06,ID,1741232402
3,3,port-aux-francais,-49.3500,70.2167,280.01,95,99,8.91,TF,1741232378
4,4,edinburgh of the seven seas,-37.0676,-12.3116,289.65,66,63,5.35,SH,1741232382


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
updated_city_data_df = city_data_df[city_data_df['Humidity'] < 75]
# city_data_df['Max Temp'] > 21, city_data_df['Wind Speed'] > 4.5, city_data_df['Cloudiness'] < 50]

# Drop any rows with null values
updated_city_data_df = updated_city_data_df.dropna()

# Display sample data
updated_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,299.11,72,17,3.32,PN,1741232385
4,4,edinburgh of the seven seas,-37.0676,-12.3116,289.65,66,63,5.35,SH,1741232382
7,7,thompson,55.7435,-97.8558,258.24,72,75,7.20,CA,1741232539
13,13,tura,25.5198,90.2201,300.80,16,0,1.51,IN,1741232546
15,15,kristiansund,63.1115,7.7320,283.51,57,75,18.52,NO,1741232548
17,17,touros,-5.1989,-35.4608,300.08,74,100,6.32,BR,1741232551
19,19,blackmans bay,-43.0167,147.3167,290.78,59,49,3.20,AU,1741232361
24,24,ahmadpur,18.7000,76.9333,298.28,11,5,3.76,IN,1741232560
27,27,ilulissat,69.2167,-51.1000,256.16,71,37,3.09,GL,1741232304
29,29,wamba,8.9333,8.6000,295.98,23,89,1.07,NG,1741232566


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = updated_city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,72,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,66,
7,thompson,CA,55.7435,-97.8558,72,
13,tura,IN,25.5198,90.2201,16,
15,kristiansund,NO,63.1115,7.7320,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {"apiKey": geoapify_key,
    "categories": categories,
    "limit": limit
}
params["filter"] = f"circle:{lon},{lat},{radius}"
params["bias"] = f"proximity:{lon},{lat}"

base_url = "https://api.geoapify.com/v2/places"
response = requests.get(base_url, params=params)

print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

Status Code: 200
Response Content: {"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Ibis Styles","country":"Australia","country_code":"au","state":"Queensland","city":"Mount Isa","municipality":"Mount Isa City","postcode":"4825","district":"Mount Isa City","suburb":"Mount Isa City","street":"Camooweal Street","lon":139.49266533264097,"lat":-20.72598125,"state_code":"QLD","formatted":"Ibis Styles, Camooweal Street, Mount Isa City QLD 4825, Australia","address_line1":"Ibis Styles","address_line2":"Camooweal Street, Mount Isa City QLD 4825, Australia","categories":["accommodation","accommodation.hotel","building","building.accommodation"],"details":["details"],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database License","url":"https://www.openstreetmap.org/copyright","raw":{"name":"Ibis Styles","brand":"Ibis Styles","osm_id":273043502,"tourism":"hotel","building":"yes","osm_type":"w","brand:wikidat

In [28]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params = {"apiKey": geoapify_key,
    "categories": categories,
    "limit": limit,
    "filter": f"circle:{lon},{lat},{radius}",
    "bias": f"proximity:{lon},{lat}"
    }
    # params["filter"] = f"circle:{lon},{lat},{radius}"
    # params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.request("GET", base_url, params=params)

    # Convert the API response to JSON format
    
    if response.status_code == 200:

        name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    else:
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
tura - nearest hotel: Hotel Polo Orchid
kristiansund - nearest hotel: Comfort Hotel Fosna
touros - nearest hotel: Pousada Atlântico
blackmans bay - nearest hotel: Villa Howden
ahmadpur - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
wamba - nearest hotel: No hotel found
shambu - nearest hotel: ዋቤ ሆቴል
broken hill - nearest hotel: Ibis Styles
ulsteinvik - nearest hotel: Quality Hotel Ulstein
uturoa - nearest hotel: Hawaiki Nui hotel
georgetown - nearest hotel: Page 63 hostel
margaret river - nearest hotel: Margaret River Hotel
atafu village - nearest hotel: No hotel found
trindade - nearest hotel: Hotel Liguori
khash - nearest hotel: No hotel found
novyy turtkul' - nearest hotel: To'rtku'l miymanxanasi
kuching - nearest hotel: Hotel Samudera
alice springs - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,72,No hotel found
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,66,No hotel found
7,thompson,CA,55.7435,-97.8558,72,Thompson Inn
13,tura,IN,25.5198,90.2201,16,Hotel Polo Orchid
15,kristiansund,NO,63.1115,7.7320,57,Comfort Hotel Fosna
17,touros,BR,-5.1989,-35.4608,74,Pousada Atlântico
19,blackmans bay,AU,-43.0167,147.3167,59,Villa Howden
24,ahmadpur,IN,18.7000,76.9333,11,No hotel found
27,ilulissat,GL,69.2167,-51.1000,71,Best Western Plus Hotel Ilulissat
29,wamba,NG,8.9333,8.6000,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)